In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
    output_df = input_df.withColumn("ingestion_date", current_timestamp())
    return output_df

In [0]:
def re_arrange_partition_column(input_df,input_column_name):
    columns_list = []
    for column_name in input_df.schema.names:
        if (column_name != input_column_name):
            columns_list.append(column_name)
    columns_list.append(input_column_name)
    return input_df.select(columns_list)

In [0]:
# discontinued
# def partition_overwrite(type_load, data_df, database_name, table_name, data_format, partition_name = ""):
#     full_database_name = f"{database_name}.{table_name}"
#     if(partition_name != ""):
#         data_df = re_arrange_partition_column(data_df,partition_name) 
        
#     spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
    
#     # full load
#     if(type_load == "F"):
#         if(partition_name != ""):
#             data_df.write.mode("overwrite").partitionBy(partition_name).format(data_format).saveAsTable(full_database_name)
#         else:
#             data_df.write.mode("overwrite").format(data_format).saveAsTable(full_database_name)
#     # incremental load
#     if(type_load == "I"):
#         if (spark._jsparkSession.catalog().tableExists(full_database_name)):
#             data_df.write.mode("overwrite").insertInto(full_database_name)
#         else:
#             if(partition_name != ""):
#                 data_df.write.mode("overwrite").partitionBy(partition_name).format(data_format).saveAsTable(full_database_name)
#             else:
#                 data_df.write.mode("overwrite").format(data_format).saveAsTable(full_database_name)

In [0]:
def full_write_delta_data(input_df, database_name, table_name, partition_column = ""):
    if(partition_column != ""):
        input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{database_name}.{table_name}")
    else:
        input_df.write.mode("overwrite").format("delta").saveAsTable(f"{database_name}.{table_name}")

In [0]:
from delta.tables import DeltaTable

def merge_delta_data(input_df, database_name, table_name, folder_path, merge_condition, partition_column):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning","true")
    
    if (spark._jsparkSession.catalog().tableExists(f"{database_name}.{table_name}")):

        deltaTable = DeltaTable.forPath(spark, f"{folder_path}/{table_name}")

        deltaTable.alias('target').merge(
            input_df.alias('source'),
            merge_condition) \
          .whenMatchedUpdateAll() \
          .whenNotMatchedInsertAll() \
          .execute()
    else:
        input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{database_name}.{table_name}")

In [0]:
def df_column_to_list(input_df,column_name):
    df_row_list = input_df.select(column_name) \
                          .distinct() \
                          .collect()
    column_value_list = [row[column_name] for row in df_row_list]
    return column_value_list